In [1]:
import icecube
from icecube.icetray.i3logging import log_info, log_fatal
from icecube import dataclasses, dataio, icetray, simclasses, MuonGun, hdfwriter
import matplotlib.pyplot as plt
import numpy as np
from I3Tray import I3Tray
import glob
from scipy.interpolate import CubicSpline

**Utility functions:**

In [2]:
#Function to read the GCD file and make the extruded polygon which
#defines the edge of the in-ice array
def MakeSurface(gcdName, padding):
    file = dataio.I3File(gcdName, "r")
    frame = file.pop_frame()
    while not "I3Geometry" in frame:
        frame = file.pop_frame()
    geometry = frame["I3Geometry"]
    xyList = []
    zmax = -1e100
    zmin = 1e100
    step = int(len(geometry.omgeo.keys())/10)
    print("Loading the DOM locations from the GCD file")
    for i, key in enumerate(geometry.omgeo.keys()):
        if i % step == 0:
            print( "{0}/{1} = {2}%".format(i,len(geometry.omgeo.keys()), int(round(i/len(geometry.omgeo.keys())*100))))
            
        if key.om in [61, 62, 63, 64] and key.string <= 81: #Remove IT...
            continue

        pos = geometry.omgeo[key].position

        if pos.z > 1500:
            continue
            
        xyList.append(pos)
        i+=1
    
    return MuonGun.ExtrudedPolygon(xyList, padding) 

def get_position_along_track(particle, length):
    return particle.pos + dataclasses.I3Position(length, particle.dir.theta, particle.dir.phi, dataclasses.I3Position.sph)


**Interpolation of neutrino interaction probabilities from** http://www.nature.com/doifinder/10.1038/nature24459

In [3]:
E_nu = [139.3307426423607, 245.3934466093052, 427.88747974059413, 708.1315242487499, 1222.0509323577705, 2672.7624459727444,
        3306.910892645023, 5061.7909183696775, 10287.314726142255, 28446.82247513599, 66737.28730628472, 163985.18407345077,
        632109.9108011357, 856943.3563562501, 1863162.1062250915]
sigma_nu = [9.38045658e-37, 1.61676917e-36, 2.73988855e-36, 4.43237880e-36, 7.34739264e-36, 1.44747135e-35, 1.72285975e-35,
            2.38716559e-35,3.96252123e-35, 7.97213420e-35, 1.40340551e-34, 2.53738392e-34, 6.26907360e-34, 7.79360007e-34, 1.31111407e-33]
E_nu_bar = [149.62511375815114, 242.64629802906907, 433.23537305604464, 708.1315242487499, 1222.0509323577705, 2672.7624459727444,
            3306.910892645023, 5061.7909183696775, 10287.314726142255, 28446.82247513599, 66737.28730628472, 163985.18407345077,
            615070.7489250309, 856943.3563562501, 1863162.1062250915]
sigma_nu_bar = [5.33231887e-37, 8.64739482e-37, 1.55287247e-36, 2.55277043e-36, 4.32998293e-36, 9.08519251e-36, 1.09686016e-35, 
                1.56436419e-35, 2.81948626e-35, 6.21615750e-35, 1.18369427e-34, 2.19996584e-34, 5.72041108e-34, 7.08829690e-34, 1.23444074e-33]
# callable functions
func_sigma_nu = CubicSpline(E_nu, sigma_nu)
func_sigma_nu_bar = CubicSpline(E_nu_bar, sigma_nu_bar)

**Class that selects background candidates**

In [4]:
# Module to select background candidate events
class BackgroundCandidateSelector(icetray.I3Module):
    """
        This module selects events that can mimic Long Lived Particle (LLP) signature.
        A stopping muon followed by a collinear neutrino interacting some distance after
        the stop point can give a similar track gap signature as LLPs. This module selects
        events with one or many stopping muons, that aren't too close to the detector boundary,
        and that are accompanied by a collinear neutrino(s).
    """
    def __init__(self,ctx):
        icetray.I3Module.__init__(self,ctx)

        self.gcdFile = ""
        self.AddParameter("GCDFile", "GCD file which defines the in-ice volume", self.gcdFile)

        self.padding = 0. * icetray.I3Units.m # default no padding
        self.AddParameter("Padding", "", self.padding)

    def Configure(self):

        # track gap properties
        self.minimum_stop_length = 50. # minimum muon length inside detector
        self.minimum_exit_length = 50. # minimum distance from neutrino interaction to detector edge
        self.minimum_track_gap = 30. # shortest visible track gap
        self.max_angle_diff = 0.087 # 5 deg separation between nu and mu maximum (both phi and theta)
        
        # padding for gcd file, use 0 by default
        self.padding = self.GetParameter("Padding")

        # create surface for detector volume
        self.gcdFile = self.GetParameter("GCDFile")
        if self.gcdFile != "":
            self.surface = MakeSurface(self.gcdFile, self.padding)
        else:
            self.surface = MuonGun.Cylinder(1000,500) # approximate detector volume
    
        
    def DAQ(self, frame):
        # stopping muon/bundle should not be too close to boundary
        self.longest_distance_from_boundary = 0 # longest distance from entering volume
        self.shortest_distance_to_boundary = 999999 # shortest distance to exiting volume
        self.longest_stopping_muon = None # will hold the muon that traveled furthest in the volume
        
        if not self.CheckCollinearNeutrino(frame):
            return False
        if not self.SelectStoppingMuon(frame):
            return False
        # write probability to file
        self.CalculateTrackGapProbability(frame)
        # good background candidate
        self.PushFrame(frame)
        return True # TODO: is this return True needed?
    
    def NeutrinoInteractionProbability(self, l1, l2, energy, species):
        """ probability for neutrino to interact between distance l1 and l2 """
        rhoH20 = 0.92 # g/cm3 for -30C ice
        NA = 6.02214076E23 # mol
        n = rhoH20 * NA # nucleon density (implicit 1 g/mol for nucleons)

        if species == 12 or species == 14 or species == 16:
            sigma = func_sigma_nu(energy) # neutrino nucleon cross section
        if species == -12 or species == -14 or species == -16:
            sigma = func_sigma_nu_bar(energy)

        # length in cm
        interaction_prob = np.exp(-(l1*100)*n*sigma) - np.exp(-(l2*100)*n*sigma) # portion of interaction pdf between l1 and l2
        return interaction_prob
    
    def CalculateTrackGapProbability(self, frame):
        # lengths for probability of neutrino interaction
        l1 = self.longest_stopping_muon.length + self.minimum_track_gap
        l2 = self.longest_stopping_muon.length + self.shortest_distance_to_boundary - self.minimum_exit_length
        interaction_prob = 0
        for p in frame["I3MCTree_preMuonProp"]:
            if p.is_neutrino:
                if abs(p.dir.zenith - self.longest_stopping_muon.dir.zenith) < self.max_angle_diff and abs(p.dir.azimuth - self.longest_stopping_muon.dir.azimuth) < self.max_angle_diff:
                    interaction_prob += self.NeutrinoInteractionProbability(l1, l2, p.energy, p.type)

        # save information to frame
        interaction_map = dataclasses.I3MapStringDouble()
        interaction_map["NeutrinoInteractionProbability"] = interaction_prob
        interaction_map["MinLength"] = l1
        interaction_map["MaxLength"] = l2
        interaction_map["AvailableInteractionLength"] = l2 - l1
        interaction_map["StoppingMuonMinorID"] = float(self.longest_stopping_muon.id.minorID)
        frame["LLPBackgroundProbability"] = interaction_map
        
    def CheckCollinearNeutrino(self, frame):
        # TODO: update to check the collinear part, not only neutrino type
        mctree = frame['SignalI3MCTree']
        primary = mctree.get_head()
        children = mctree.children(primary)
        has_collinear_neutrino = False
        for c in children:
            if c.is_neutrino:
                has_collinear_neutrino = True
                break
        return has_collinear_neutrino
    
    def SelectStoppingMuon(self, frame):
        through_going = False
        n_stop = 0
        for track in MuonGun.Track.harvest(frame['SignalI3MCTree'], frame['MMCTrackList']):
            intersections = self.surface.intersection(track.pos, track.dir)
            #print(get_position_along_track(track, intersections.first), get_position_along_track(track, intersections.second))
            if intersections.first >= track.length: # Stop before volume
                continue
            elif intersections.second >= track.length: # Stop inside volume
                n_stop += 1
                # update longest stopping muon lengths if applicable
                from_boundary = (track.length - intersections.first)
                to_boundary = (intersections.second - track.length)
                if from_boundary > self.longest_distance_from_boundary:
                    self.longest_distance_from_boundary = from_boundary
                if to_boundary < self.shortest_distance_to_boundary:
                    self.shortest_distance_to_boundary = to_boundary
                    self.longest_stopping_muon = track

            elif track.length > intersections.second: # Stop after volume
                through_going = True # one of these ruins the whole event
                break 
            else: # Not entering volume
                continue

        # avoid stopping muon/bundles to close to boundaries
        distance_check = self.shortest_distance_to_boundary > (self.minimum_exit_length + self.minimum_track_gap) and self.longest_distance_from_boundary > self.minimum_stop_length
        # candidates have no through going muons and a stopping muon not too close to boundary
        good_event = n_stop >= 1 and (not through_going) and distance_check
        return good_event # true or false
    
    def Finish(self):
        self.outputFile.close()

In [ ]:
filelist = list(glob.glob("/data/sim/IceCube/2020/generated/CORSIKA-in-ice/20904/0198000-0198999/detector/IC86.2020_corsika.020904.198*.i3.zst"),)
n_files = 10
filelist = filelist[0:n_files] # how many files to use?
print("Number of files ", len(filelist))
gcdfile = "gcdfile.i3.gz"
output_name = "Selected_bkg_candidates_"+str(n_files)+"files.i3.gz"

tray = I3Tray()

tray.Add("I3Reader", filenamelist=filelist)
tray.Add(BackgroundCandidateSelector, GCDFile = gcdfile)
tray.Add("I3Writer", filename=output_name)

tray.Execute()

In [ ]:

tray = I3Tray()
tray.Add("I3Reader", FileNameList=[output_name])
tray.Add(
    hdfwriter.I3SimHDFWriter,
    keys=["PolyplopiaPrimary", "CorsikaWeightMap", "LLPBackgroundProbability"],
    output="Selected_bkg_candidates_"+str(n_files)+"files.hdf5",
)

tray.Execute()

In [ ]:
# plot the muon energy at surface? incomplete and incorrect

infile = dataio.I3File(output_name, "r")

surface = MakeSurface("gcdfile.i3.gz", 0)
energies_best_muon = []
while infile.more():
    frame = infile.pop_daq()
    bkginfo = frame["LLPBackgroundProbability"]
    muon_id = bkginfo["StoppingMuonMinorID"]
    for track in MuonGun.Track.harvest(frame['SignalI3MCTree'], frame['MMCTrackList']):
        if track.id.minorID == muon_id:
            intersections = surface.intersection(track.pos, track.dir)
            energies_best_muon = track.get_energy(intersections.first)
            track.get_energy(track.length - 50)
            
plt.figure()
plt.hist(energies_best_muon)
plt.show()